In [1]:
import numpy as np
import random
from scipy import ndimage, misc
import copy
import time
from deap import base
from deap import creator
from deap import tools
import game


In [2]:
n_weights = 9
population_size = 20
crossover_probability = 0.5
mutation_probability = 0.1

moves = [0,1,2,3]
directions = {
        0: "down",
        1: "up",
        2: "left",
        3: "right"
    }

gameIterations = 1000

In [3]:
''' DEAP SETUP PT 1'''

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("GameInstance", game.Game)
creator.create("Individual", list, 
                fitness=creator.FitnessMax, 
                gameInstance=creator.GameInstance)

toolbox = base.Toolbox()
toolbox.register("attr_bool", np.random.uniform, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, n_weights)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [4]:
''' Example '''

# Create individual
ind = toolbox.individual()
print(ind) # 10 weights between 0-1

# Game within Individual
print("---")
print(ind.gameInstance.grid)

# Make a move in the game
ind.gameInstance.move(0) # 0 => down 
print("---")
print(ind.gameInstance.grid)

[0.9648246340867026, 0.8275996863806304, 0.09800680443710741, 0.6851485790333325, 0.5835882138170128, 0.08209327751418338, 0.621680499051048, 0.8510440326323003, 0.44121472003306883]
---
[[0 0 0 0]
 [0 2 0 0]
 [0 0 0 0]
 [0 0 0 2]]
---
[[2 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 2 0 2]]


In [66]:
''' F1 '''
# f1 : Feature f1 is 1, if tile with maximum value is on one of the corners in a 
# given state. Otherwise, it is 0.
def f1(newstate):
    flat_grid = newstate.flatten()

    maxvalue = max(flat_grid)
    for i,value in enumerate(flat_grid):
        if value == maxvalue and i in [0,3,11,15]:
            return 1
    return 0

''' F2 '''
# Feature f2 is 1, if second or third highest value tile is next to highest
# value tile. Otherwise, it is 0.
def f2(newstate):
    maxval = max(newstate.flatten())
    if maxval==2: return 0

    if maxval==4:
        maxvalues = [4,2]
    else:
        maxvalues = [maxval, maxval//2, maxval//4]

    for row in range(4):
        for column in range(4):
            val = newstate[row,column]
            if val == maxval:
                # # Check above
                # if column-1 >= 0 and individual.gameInstance.grid[row,column-1] in maxvalues:
                #     return 1

                # Check right
                if row+1 <= 3 and newstate[row+1,column] in maxvalues:
                    return 1

                # # Check below
                # if column+1 <= 3 and individual.gameInstance.grid[row,column+1] in maxvalues:
                #     return 1

                # Check left
                if row-1 >= 0 and newstate[row-1,column] in maxvalues:
                    return 1
    return 0

''' F3 '''
# Sum of total changes 
def f3(individual, newstate):
    return 16-sum(individual.gameInstance.grid.flatten() == newstate.flatten())

''' F4 '''
# Number of non-empty cells in the given state of the board.
def f4(newstate):
    return sum([i for i in newstate.flatten() if i != 0])

''' F5'''
# Modified version of F5 : how many selected cells have equal neighbors.
def f5(newstate,selected_cells=[[0,0],[0,3],[1,1],[2,2],[3,0]]):
    counter = 0
    for cell in selected_cells:
        row = cell[0]
        column = cell[1]
        val = newstate[row,column] # Important: newstate can not be a list object
        # Check below
        if row+1 <= 3 and newstate[row+1,column] == val:
            counter += 1
        #Check the above
        if row-1 >= 0 and newstate[row-1,column] == val:
            counter += 1
        #Check left
        if column-1 >= 0 and newstate[row,column-1] == val:
            counter += 1
        #Check right
        if column+1 <= 3 and newstate[row,column+1] == val:
            counter +=1
    return counter
    
''' F6 '''
# Check if the given cells have equal neighbors, reuse function f5_modified
def f6(newstate):
    selected_cells = [[2,3],[3,2],[0,1],[0,2],[1,1],[2,0],[1,3],[3,1]]
    return f5(newstate,selected_cells)


#1 if the row/column containing highest and 2nd highest value 
# tile does not have empty cell return 1
def f7(newstate):

    # Finding higest value
    maxvalue = max(newstate.flatten())
    maxIndex = np.where(newstate.flatten()==maxvalue)[-1]
    row, column = maxIndex//4, maxIndex-(maxIndex//4)*4
    if newstate[row].any() == 0 or newstate.T[column].any() == 0:
        return 0

    # Finding second highest value
    if maxvalue != 2: 
        secondIndex = np.where(newstate.flatten()==maxvalue//2)[-1]
        row, column = secondIndex//4, secondIndex-(secondIndex//4)*4
        if newstate[row].any() == 0 or newstate.T[column].any() == 0:
            return 0
    return 1




In [67]:
ind = toolbox.individual()
state = ind.gameInstance.project(0)
print(state)
f7(state)

[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 4 0 0]]


0

In [65]:
# The state evaluation function
def stateEvaluation(ind, state):
    E = ind[0]*f1(state) + ind[1]*f2(state) + ind[2]*f3(ind, state) + ind[3]*f4(state) + ind[4]*f5(state) + ind[5]*f6(state) + ind[6]*f7(state)
    return E


In [16]:
ind = toolbox.individual()

statesEvaluation = []

# Generate possible future states
for move in moves:
    state = ind.gameInstance.project(move)
    E = stateEvaluation(ind, state)
    statesEvaluation.append(E)

chosenMove = moves[np.argmax(statesEvaluation)]


In [17]:
# Game Evaluation Function
def gameEvaluation(moves, individual, gameIterations):

    # Play game for 1000 moves
    for iter in range(gameIterations):
        statesEvaluation = []

        # Evaluate all possible moves (projections)
        for move in moves:
            state = ind.gameInstance.project(move)
            if np.array_equal(state, individual.gameInstance.grid): E = 0
            else: E = stateEvaluation(individual, state)
            statesEvaluation.append(E)
            
        # Choose the best possible move with maximum E
        chosenMove = moves[np.argmax(statesEvaluation)]

        try: individual.gameInstance.move(chosenMove)
        except: 
            fitness = (max(individual.gameInstance.grid.flatten()) + sum([i for i in individual.gameInstance.grid.flatten() if i == 0]),)
            individual.gameInstance.start()
            # Return fitness
            return fitness

    # Return the fitness, 
    # Maximum value and number of zeros after 1000 game moves
    fitness = (max(individual.gameInstance.grid.flatten()) + sum([i for i in individual.gameInstance.grid.flatten() if i == 0]),)
    individual.gameInstance.start()
    return fitness

In [18]:
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=mutation_probability)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("select", tools.selTournament, k=population_size)
toolbox.register("gameEvaluation", gameEvaluation, gameIterations=gameIterations, moves=moves)

In [19]:
def main():
    best = []
    # Create the population
    population = toolbox.population(population_size)

    # Evaluate
    for individual in population:
        individual.fitness.values = toolbox.gameEvaluation(individual=individual)

    # Keep track of the no. of generations
    generations = 0
    
    # Begin evolution
    while generations < 100:
                
        # Selection
        #selected = toolbox.select(population, len(population))
        offspring = [toolbox.clone(ind) for ind in population]
        
        # Crossover
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random()> crossover_probability:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        # Mutation
        for mutant in offspring:
            toolbox.mutate(mutant)
            del mutant.fitness.values

        # Evaluate new individuals with no fitness
        for individual in offspring:
            if not individual.fitness.valid:
                individual.fitness.values = toolbox.gameEvaluation(individual=individual)
   
        population = tools.selBest(offspring+population,k=2) + toolbox.select(offspring, tournsize=population_size-2)
        
        best.append(tools.selBest(population, k=1)[0])
        average = sum([ind.fitness.values[0] for ind in population])/len(population)
        print(best[-1].fitness.values[0], average, best[-1].gameInstance.highscore)
        
        # Increment the generation counter
        generations += 1    
    
    return tools.selBest(population,k=1)
    

In [20]:
run_algorithm = main()

128.0 64.72727272727273 128
128.0 128.0 128
128.0 107.63636363636364 128
128.0 101.81818181818181 128
128.0 98.9090909090909 128
256.0 194.9090909090909 256
256.0 238.54545454545453 256
256.0 119.27272727272727 256
256.0 212.36363636363637 256
256.0 215.27272727272728 256
256.0 139.63636363636363 256
256.0 215.27272727272728 256
256.0 197.8181818181818 256
256.0 139.63636363636363 256
256.0 133.8181818181818 256
256.0 226.9090909090909 256
256.0 136.72727272727272 256
256.0 194.9090909090909 256
256.0 139.63636363636363 256
256.0 232.72727272727272 256
256.0 232.72727272727272 256
256.0 139.63636363636363 256
256.0 139.63636363636363 256
256.0 133.8181818181818 256
256.0 212.36363636363637 256
256.0 174.54545454545453 256
256.0 133.8181818181818 256
256.0 238.54545454545453 256
256.0 209.45454545454547 256
256.0 133.8181818181818 256
256.0 133.8181818181818 256
256.0 139.63636363636363 256
256.0 139.63636363636363 256
256.0 139.63636363636363 256
256.0 139.63636363636363 256
256.0 139.

In [ ]:
best_individual = run_algorithm[0]

best_individual.gameInstance.start()
# Play game with best individual
for iter in range(100):
    statesEvaluation = []

    # Evaluate all possible moves (projections)
    for move in moves:
        state = best_individual.gameInstance.project(move)
        if np.array_equal(state, best_individual.gameInstance.grid): E = 0
        else: E = stateEvaluation(best_individual, state)
        statesEvaluation.append(E)
        
    # Choose the best possible move with maximum E
    chosenMove = moves[np.argmax(statesEvaluation)]
    try: best_individual.gameInstance.move(chosenMove)
    except: 
        print(best_individual.gameInstance.grid)
        break

print(best_individual.gameInstance.grid)
